In [1]:
%matplotlib inline
import utils; #reload(utils)
from utils import *
from __future__ import division, print_function
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras.utils.np_utils as kutils

Using Theano backend.


In [2]:
train=pd.read_csv('digit/data/train.csv')
test=pd.read_csv('digit/data/test.csv')


In [3]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.shape

(28000, 784)

In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(train,train['label'],test_size=0.20)


In [6]:
train_label=X_train['label']
del X_train['label']
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
10839,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
val_label=X_test['label']
del X_test['label']



X_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
28813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X=np.array(X_train)
Va=np.array(X_test)
te=np.array(test)
test.shape

(28000, 784)

In [97]:



trainX = X.reshape(X.shape[0], 28, 28,1)
trainX = trainX.astype(float)
trainX /= 255.0

train_label


valX = Va.reshape(Va.shape[0], 28, 28,1)
valX = valX.astype(float)
valX /= 255.0

val_label

testX = te.reshape(te.shape[0], 28, 28,1)
testX = testX.astype(float)
testX /= 255.0



In [59]:
print (trainX.shape)
print (valX.shape)
print (testX.shape)

(33600, 28, 28, 1)
(8400, 28, 28, 1)
(28000, 28, 28, 1)


In [53]:
y_train=np.array(Y_train)
y_test=np.array(Y_test)

y_train = onehot(y_train)
y_val = onehot(y_test)


print (y_train.shape)
print (y_val.shape)

(33600, 10)
(8400, 10)


In [10]:
mean_px = trainX.mean().astype(np.float32)
std_px = trainX.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [71]:
ml1=get_lin_model()

In [54]:
#gen = image.ImageDataGenerator()

gen = image.ImageDataGenerator(rotation_range=20, width_shift_range=0.1, 
       height_shift_range=0.1,  shear_range=0.15,
       vertical_flip=True, 
       horizontal_flip=True)


batches = gen.flow(trainX, y_train, batch_size=64)
val_batches = gen.flow(valX, y_val, batch_size=64)
#test_batches = gen.flow(testX, y_test, batch_size=64)

In [73]:
ml1.fit_generator(batches,batches.N,nb_epoch=1,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/1
33600/33600 [==============================] - 10s - loss: 0.5233 - acc: 0.8426 - val_loss: 0.3179 - val_acc: 0.9086


In [75]:
def get_lin_model1():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [76]:
ml2=get_lin_model1()

In [78]:
ml2.fit_generator(batches,batches.N,nb_epoch=4,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/4
33600/33600 [==============================] - 13s - loss: 0.0975 - acc: 0.9702 - val_loss: 0.1119 - val_acc: 0.9639
Epoch 2/4
33600/33600 [==============================] - 13s - loss: 0.0577 - acc: 0.9823 - val_loss: 0.1174 - val_acc: 0.9626
Epoch 3/4
33600/33600 [==============================] - 13s - loss: 0.0369 - acc: 0.9886 - val_loss: 0.1203 - val_acc: 0.9657
Epoch 4/4
33600/33600 [==============================] - 13s - loss: 0.0282 - acc: 0.9917 - val_loss: 0.1068 - val_acc: 0.9699


In [84]:
ml2.optimizer.lr=0.1

In [85]:
ml2.fit_generator(batches,batches.N,nb_epoch=4,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/4
33600/33600 [==============================] - 14s - loss: 0.0158 - acc: 0.9949 - val_loss: 0.1239 - val_acc: 0.9751
Epoch 2/4
33600/33600 [==============================] - 13s - loss: 0.0092 - acc: 0.9968 - val_loss: 0.1131 - val_acc: 0.9787
Epoch 3/4
33600/33600 [==============================] - 13s - loss: 0.0147 - acc: 0.9949 - val_loss: 0.1604 - val_acc: 0.9714
Epoch 4/4
33600/33600 [==============================] - 13s - loss: 0.0174 - acc: 0.9945 - val_loss: 0.1352 - val_acc: 0.9743


In [12]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [87]:
model=get_model()

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [88]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-3.13--generic-x86_64-with-debian-jessie-sid-x86_64-3.5.2-64/lock_dir/lock


Epoch 1/1
33600/33600 [==============================] - 104s - loss: 0.1514 - acc: 0.9537 - val_loss: 0.0720 - val_acc: 0.9773


In [89]:
model.optimizer.lr=0.1

In [90]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
33600/33600 [==============================] - 107s - loss: 0.0412 - acc: 0.9870 - val_loss: 0.0366 - val_acc: 0.9892


In [91]:
model.optimizer.lr=0.01

In [92]:
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/8
33600/33600 [==============================] - 108s - loss: 0.0274 - acc: 0.9913 - val_loss: 0.0425 - val_acc: 0.9864
Epoch 2/8
33600/33600 [==============================] - 108s - loss: 0.0197 - acc: 0.9937 - val_loss: 0.0443 - val_acc: 0.9887
Epoch 3/8
33600/33600 [==============================] - 109s - loss: 0.0164 - acc: 0.9944 - val_loss: 0.0433 - val_acc: 0.9880
Epoch 4/8
33600/33600 [==============================] - 108s - loss: 0.0138 - acc: 0.9956 - val_loss: 0.0463 - val_acc: 0.9849
Epoch 5/8
33600/33600 [==============================] - 108s - loss: 0.0122 - acc: 0.9960 - val_loss: 0.0394 - val_acc: 0.9907
Epoch 6/8
33600/33600 [==============================] - 108s - loss: 0.0093 - acc: 0.9970 - val_loss: 0.0749 - val_acc: 0.9813
Epoch 7/8
33600/33600 [==============================] - 107s - loss: 0.0131 - acc: 0.9960 - val_loss: 0.0555 - val_acc: 0.9869
Epoch 8/8
33600/33600 [==============================] - 107s - loss: 0.0089 - acc: 0.9972 - val_loss: 0

In [13]:
model=get_model()

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [11]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(trainX, y_train, batch_size=64)
val_batches = gen.flow(valX, y_val, batch_size=64)

In [15]:
model.fit_generator(batches,batches.N,nb_epoch=1,validation_data=val_batches,nb_val_samples=val_batches.N)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '18085' (I am process '8391')


Epoch 1/1
33600/33600 [==============================] - 106s - loss: 0.2839 - acc: 0.9085 - val_loss: 0.0999 - val_acc: 0.9687


In [16]:
model.optimizer.lr=0.01

In [18]:
model.fit_generator(batches,batches.N,nb_epoch=7,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/7
33600/33600 [==============================] - 111s - loss: 0.0441 - acc: 0.9864 - val_loss: 0.0528 - val_acc: 0.9843
Epoch 2/7
33600/33600 [==============================] - 111s - loss: 0.0449 - acc: 0.9864 - val_loss: 0.0476 - val_acc: 0.9861
Epoch 3/7
33600/33600 [==============================] - 112s - loss: 0.0411 - acc: 0.9873 - val_loss: 0.0468 - val_acc: 0.9844
Epoch 4/7
33600/33600 [==============================] - 111s - loss: 0.0401 - acc: 0.9877 - val_loss: 0.0491 - val_acc: 0.9858
Epoch 5/7
33600/33600 [==============================] - 111s - loss: 0.0347 - acc: 0.9890 - val_loss: 0.0595 - val_acc: 0.9804
Epoch 6/7
33600/33600 [==============================] - 111s - loss: 0.0338 - acc: 0.9893 - val_loss: 0.0374 - val_acc: 0.9898
Epoch 7/7
33600/33600 [==============================] - 110s - loss: 0.0339 - acc: 0.9897 - val_loss: 0.0357 - val_acc: 0.9896


In [19]:
model.optimizer.lr=0.001

In [20]:
model.fit_generator(batches,batches.N,nb_epoch=14,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/14
33600/33600 [==============================] - 110s - loss: 0.0310 - acc: 0.9904 - val_loss: 0.0441 - val_acc: 0.9886
Epoch 2/14
33600/33600 [==============================] - 110s - loss: 0.0280 - acc: 0.9913 - val_loss: 0.0614 - val_acc: 0.9812
Epoch 3/14
33600/33600 [==============================] - 110s - loss: 0.0275 - acc: 0.9907 - val_loss: 0.0792 - val_acc: 0.9804
Epoch 4/14
33600/33600 [==============================] - 110s - loss: 0.0294 - acc: 0.9909 - val_loss: 0.0471 - val_acc: 0.9860
Epoch 5/14
33600/33600 [==============================] - 110s - loss: 0.0274 - acc: 0.9913 - val_loss: 0.0444 - val_acc: 0.9873
Epoch 6/14
33600/33600 [==============================] - 110s - loss: 0.0245 - acc: 0.9920 - val_loss: 0.0533 - val_acc: 0.9861
Epoch 7/14
33600/33600 [==============================] - 110s - loss: 0.0258 - acc: 0.9921 - val_loss: 0.0496 - val_acc: 0.9852
Epoch 8/14
33600/33600 [==============================] - 109s - loss: 0.0248 - acc: 0.9921 - val

In [21]:
model.optimizer.lr=0.0001

In [22]:
model.fit_generator(batches,batches.N,nb_epoch=10,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/10
33600/33600 [==============================] - 110s - loss: 0.0202 - acc: 0.9937 - val_loss: 0.0395 - val_acc: 0.9885
Epoch 2/10
33600/33600 [==============================] - 110s - loss: 0.0193 - acc: 0.9934 - val_loss: 0.0392 - val_acc: 0.9900
Epoch 3/10
33600/33600 [==============================] - 109s - loss: 0.0212 - acc: 0.9932 - val_loss: 0.0462 - val_acc: 0.9863
Epoch 4/10
33600/33600 [==============================] - 109s - loss: 0.0192 - acc: 0.9940 - val_loss: 0.0521 - val_acc: 0.9875
Epoch 5/10
33600/33600 [==============================] - 109s - loss: 0.0181 - acc: 0.9940 - val_loss: 0.0405 - val_acc: 0.9910
Epoch 6/10
33600/33600 [==============================] - 109s - loss: 0.0192 - acc: 0.9941 - val_loss: 0.0278 - val_acc: 0.9917
Epoch 7/10
33600/33600 [==============================] - 110s - loss: 0.0189 - acc: 0.9942 - val_loss: 0.0408 - val_acc: 0.9902
Epoch 8/10
33600/33600 [==============================] - 108s - loss: 0.0168 - acc: 0.9948 - val

In [12]:
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [24]:
model=get_model_bn()

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [25]:
        model.fit_generator(batches,batches.N,nb_epoch=1,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/1
33600/33600 [==============================] - 148s - loss: 0.2048 - acc: 0.9350 - val_loss: 0.1963 - val_acc: 0.9419


In [26]:
model.optimizer.lr=0.1

In [27]:
    model.fit_generator(batches,batches.N,nb_epoch=4,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/4
33600/33600 [==============================] - 150s - loss: 0.0820 - acc: 0.9735 - val_loss: 0.0704 - val_acc: 0.9788
Epoch 2/4
33600/33600 [==============================] - 148s - loss: 0.0731 - acc: 0.9768 - val_loss: 0.0626 - val_acc: 0.9825
Epoch 3/4
33600/33600 [==============================] - 147s - loss: 0.0575 - acc: 0.9815 - val_loss: 0.0490 - val_acc: 0.9857
Epoch 4/4
33600/33600 [==============================] - 147s - loss: 0.0534 - acc: 0.9838 - val_loss: 0.0694 - val_acc: 0.9781


In [29]:
model.optimizer.lr=0.01

In [30]:
model.fit_generator(batches,batches.N,nb_epoch=12,validation_data=val_batches,nb_val_samples=val_batches.N)

Epoch 1/12
33600/33600 [==============================] - 148s - loss: 0.0505 - acc: 0.9841 - val_loss: 0.0629 - val_acc: 0.9800
Epoch 2/12
33600/33600 [==============================] - 147s - loss: 0.0443 - acc: 0.9854 - val_loss: 0.0671 - val_acc: 0.9796
Epoch 3/12
33600/33600 [==============================] - 147s - loss: 0.0436 - acc: 0.9862 - val_loss: 0.0415 - val_acc: 0.9885
Epoch 4/12
33600/33600 [==============================] - 148s - loss: 0.0422 - acc: 0.9864 - val_loss: 0.0524 - val_acc: 0.9850
Epoch 5/12
33600/33600 [==============================] - 148s - loss: 0.0392 - acc: 0.9876 - val_loss: 0.0359 - val_acc: 0.9900
Epoch 6/12
33600/33600 [==============================] - 148s - loss: 0.0374 - acc: 0.9873 - val_loss: 0.0482 - val_acc: 0.9863
Epoch 7/12
33600/33600 [==============================] - 148s - loss: 0.0353 - acc: 0.9882 - val_loss: 0.0466 - val_acc: 0.9875
Epoch 8/12
33600/33600 [==============================] - 150s - loss: 0.0361 - acc: 0.9882 - val

In [13]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches,batches.N,nb_epoch=1,validation_data=val_batches,nb_val_samples=val_batches.N)
    
    model.optimizer.lr=0.1
    model.fit_generator(batches,batches.N,nb_epoch=4,validation_data=val_batches,nb_val_samples=val_batches.N)
    
    model.optimizer.lr=0.01
    model.fit_generator(batches,batches.N,nb_epoch=12,validation_data=val_batches,nb_val_samples=val_batches.N)
    model.optimizer.lr=0.001
    model.fit_generator(batches,batches.N,nb_epoch=18,validation_data=val_batches,nb_val_samples=val_batches.N)
    
    return model
models = [fit_model() for i in range(6)]

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Epoch 1/1
33600/33600 [==============================] - 110s - loss: 0.2990 - acc: 0.9104 - val_loss: 0.3036 - val_acc: 0.9138
Epoch 1/4
33600/33600 [==============================] - 116s - loss: 0.1170 - acc: 0.9634 - val_loss: 0.0925 - val_acc: 0.9711
Epoch 2/4
33600/33600 [==============================] - 95s - loss: 0.0921 - acc: 0.9709 - val_loss: 0.0685 - val_acc: 0.9783
Epoch 3/4
33600/33600 [==============================] - 103s - loss: 0.0787 - acc: 0.9757 - val_loss: 0.0574 - val_acc: 0.9827
Epoch 4/4
33600/33600 [==============================] - 95s - loss: 0.0762 - acc: 0.9767 - val_loss: 0.0475 - val_acc: 0.9858
Epoch 1/12
33600/33600 [==============================] - 97s - loss: 0.0626 - acc: 0.9793 - val_loss: 0.0539 - val_acc: 0.9839
Epoch 2/12
33600/33600 [==============================] - 106s - loss: 0.0603 - acc: 0.9812 - val_loss: 0.0534 - val_acc: 0.9832
Epoch 3/12
33600/33600 [==============================] - 103s - loss: 0.0574 - acc: 0.9821 - val_loss: 0

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Epoch 1/1
33600/33600 [==============================] - 103s - loss: 0.3139 - acc: 0.9067 - val_loss: 0.1581 - val_acc: 0.9518
Epoch 1/4
33600/33600 [==============================] - 96s - loss: 0.1187 - acc: 0.9631 - val_loss: 0.0736 - val_acc: 0.9782
Epoch 2/4
33600/33600 [==============================] - 97s - loss: 0.0932 - acc: 0.9714 - val_loss: 0.0625 - val_acc: 0.9788
Epoch 3/4
33600/33600 [==============================] - 97s - loss: 0.0790 - acc: 0.9759 - val_loss: 0.0666 - val_acc: 0.9787
Epoch 4/4
33600/33600 [==============================] - 97s - loss: 0.0724 - acc: 0.9777 - val_loss: 0.0557 - val_acc: 0.9814
Epoch 1/12
33600/33600 [==============================] - 107s - loss: 0.0653 - acc: 0.9792 - val_loss: 0.0759 - val_acc: 0.9760
Epoch 2/12
33600/33600 [==============================] - 95s - loss: 0.0593 - acc: 0.9818 - val_loss: 0.0600 - val_acc: 0.9801
Epoch 3/12
33600/33600 [==============================] - 103s - loss: 0.0576 - acc: 0.9815 - val_loss: 0.0

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Epoch 1/1
33600/33600 [==============================] - 108s - loss: 0.2963 - acc: 0.9101 - val_loss: 0.1118 - val_acc: 0.9646
Epoch 1/4
33600/33600 [==============================] - 111s - loss: 0.1200 - acc: 0.9644 - val_loss: 0.0769 - val_acc: 0.9751
Epoch 2/4
33600/33600 [==============================] - 98s - loss: 0.0903 - acc: 0.9723 - val_loss: 0.0726 - val_acc: 0.9775
Epoch 3/4
33600/33600 [==============================] - 102s - loss: 0.0808 - acc: 0.9748 - val_loss: 0.0631 - val_acc: 0.9804
Epoch 4/4
33600/33600 [==============================] - 104s - loss: 0.0732 - acc: 0.9771 - val_loss: 0.0564 - val_acc: 0.9826
Epoch 1/12
33600/33600 [==============================] - 109s - loss: 0.0683 - acc: 0.9789 - val_loss: 0.0597 - val_acc: 0.9835
Epoch 2/12
33600/33600 [==============================] - 98s - loss: 0.0621 - acc: 0.9808 - val_loss: 0.0423 - val_acc: 0.9867
Epoch 3/12
33600/33600 [==============================] - 100s - loss: 0.0540 - acc: 0.9835 - val_loss: 

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Epoch 1/1
33600/33600 [==============================] - 111s - loss: 0.2974 - acc: 0.9111 - val_loss: 0.1742 - val_acc: 0.9468
Epoch 1/4
33600/33600 [==============================] - 111s - loss: 0.1108 - acc: 0.9662 - val_loss: 0.1060 - val_acc: 0.9673
Epoch 2/4
33600/33600 [==============================] - 110s - loss: 0.0957 - acc: 0.9710 - val_loss: 0.0723 - val_acc: 0.9761
Epoch 3/4
33600/33600 [==============================] - 110s - loss: 0.0773 - acc: 0.9765 - val_loss: 0.0601 - val_acc: 0.9812
Epoch 4/4
33600/33600 [==============================] - 111s - loss: 0.0706 - acc: 0.9782 - val_loss: 0.0469 - val_acc: 0.9844
Epoch 1/12
33600/33600 [==============================] - 111s - loss: 0.0653 - acc: 0.9805 - val_loss: 0.0443 - val_acc: 0.9849
Epoch 2/12
33600/33600 [==============================] - 111s - loss: 0.0632 - acc: 0.9813 - val_loss: 0.0551 - val_acc: 0.9851
Epoch 3/12
33600/33600 [==============================] - 111s - loss: 0.0582 - acc: 0.9823 - val_loss

In [16]:
models

In [18]:
path = "data/"
model_path = path + 'models/'

In [19]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [21]:
evals = np.array([m.evaluate(valX, y_val, batch_size=256) for m in models])

8400/8400 [==============================] - 6s     


In [22]:
evals.mean(axis=0)

array([ 0.0226,  0.9943])

In [23]:
all_preds = np.stack([m.predict(testX, batch_size=256) for m in models])

In [59]:
#all_preds_classes = np.stack([m.predict_classes(testX, batch_size=256) for m in models])

In [60]:
#all_preds_classes.mean(axis=0)[2]

In [26]:
avg_preds = all_preds.mean(axis=0)

In [64]:
avg_preds[0]

array([  1.0499e-10,   8.6935e-11,   1.0000e+00,   6.5346e-09,   2.4939e-10,   8.3630e-11,
         6.8040e-12,   1.0817e-08,   3.0888e-10,   1.8464e-10], dtype=float32)

In [65]:
avg_preds.shape

(28000, 10)

In [68]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
df_test=pd.DataFrame(avg_preds)

In [97]:

df_test.head()

,0,1,2,3,4,5,6,7,8,9
0,1.049885e-10,8.693482e-11,1.000000e+00,6.534650e-09,2.493860e-10,8.363002e-11,6.803990e-12,1.081714e-08,3.088789e-10,1.846404e-10
1,9.999995e-01,2.423503e-10,5.327970e-08,1.433815e-11,4.646189e-10,1.120676e-08,9.508602e-08,5.988779e-08,7.775041e-09,3.780608e-07
2,2.303405e-08,3.304975e-10,1.794845e-06,3.926413e-07,5.211029e-08,4.798926e-09,1.853113e-11,3.798635e-07,3.133607e-06,9.999942e-01
3,8.722304e-01,7.697568e-07,3.681842e-05,2.518473e-05,1.775379e-06,1.226127e-04,4.355041e-03,4.557351e-04,8.980238e-04,1.218737e-01
4,2.759671e-10,5.619114e-10,5.340772e-07,9.999683e-01,3.973704e-12,5.976860e-09,3.288208e-12,2.570735e-10,3.096805e-05,1.656182e-07


In [102]:
final=df_test.idxmax(axis=1)

In [107]:
dff=pd.DataFrame(final,columns=['Label'])

In [108]:

dff['ImageId']=test.index
dff.head()

,Label,ImageId
0,2,0
1,0,1
2,9,2
3,0,3
4,3,4


In [113]:
dff=dff[['ImageId','Label']]

In [129]:
dff['ImageId']=dff['ImageId']+1
dff.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [130]:
submission_file_name = 'submission5.csv'
np.savetxt(submission_file_name, dff,fmt='%d,%.d',header='ImageId,Label', comments='')

In [131]:
from IPython.display import FileLink
import os
os.getcwd()

'/home/ubuntu/Notebook/fastai/data'

In [126]:
%cd '/home/ubuntu/Notebook/fastai/data/'
FileLink(''+submission_file_name)

/home/ubuntu/Notebook/fastai/data


/home/ubuntu/Notebook/fastai/data/submission4.csv